In [5]:
from openai import OpenAI
import os
import json
from dotenv import load_dotenv
load_dotenv()       # load environment variables from .env file

True

In [6]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []     # Store conversation history
    # Copy user messages to input_messages in the required format
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    # Send a request to the model to generate a response
    response = client.chat.completions.create(
    model=model_name,
    messages=input_messages,
    # Temperature controls the creativity of the response
    # Lower values give more predictable answers, higher values give more creative answers
    temperature=temperature,  # 0.0 = factual, no randomness

    # top_p controls the diversity of responses. It looks at the most probable tokens
    # Lower values make the response more focused, higher values allow more randomness
    top_p=0.8,  # 0.8 = consider 80% of the most probable tokens for the response

    # max_tokens limits how long the model’s response can be
    # 2000 tokens is a large number, allowing long responses
    max_tokens=2000,  # Maximum length of the model's response (in tokens)
    ).choices[0].message.content
    return response

# Get LLM Response

In [7]:
client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_CHATBOT_URL"),   #URL for the llama model deployed on runpod
)
model_name = os.getenv("MODEL_NAME")

In [7]:
messages = [{"role": "user", "content": "What's the capital of Spain?"}]
response = get_chatbot_response(client, model_name, messages)

In [8]:
response

'The capital of Spain is Madrid.'

# Prompt Engineering

## Structured Output

In [12]:
system_prompt = """
You are a helpful assistant that answers questions about capitals of countries.

Your output should be in a structured json format exactly like the one below. Your are not allowed to write anything other than the json object:
[
{
    "country": the country that you will get the capital of,
    "capital": the capital of the country stated,
}
]
"""
messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": "What's the capital of Spain?"})
response = get_chatbot_response(client, model_name, messages)
print(response)

[
{
    "country": "Spain",
    "capital": "Madrid"
}
]


In [14]:
type(response)

str

In [15]:
json_response = json.loads(response)
json_response

[{'country': 'Spain', 'capital': 'Madrid'}]

In [16]:
type(json_response)

list

In [18]:
type(json_response[0]), json_response[0]['capital']

(dict, 'Madrid')

## Input Structuring

In [19]:
user_input = """
Give me the capital of the following countries: 
'''
1. Italy
2. Germany
3. Spain
'''
"""
messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client, model_name, messages)
print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    },
    {
        "country": "Spain",
        "capital": "Madrid"
    }
]


In [20]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'Germany', 'capital': 'Berlin'},
 {'country': 'Spain', 'capital': 'Madrid'}]

In [8]:
user_prompt = """
Calculate the result of this question: 1+3

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    "result": the final number resulted from calulating the question above
}
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "result": 4
}


In [9]:
259/2*8654+91072*33-12971

4113098.0

In [10]:
user_prompt = """
Calculate the result of this question: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    "result": the final number resulted from calulating the question above
}
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "result": 1345119
}


In [11]:
4113098.0 - 1345119     #Margin of error

2767979.0

In [13]:
user_prompt = """
Calculate the result of this question: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit by following the BE DMAS order of operations. You need to show your work and calculate each step leading to the final result. Feel free to write in free text.
    result: the final number resulted from calulating the question above
}
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client, model_name, messages)
print(response)

{
    "steps": "First, calculate the division: 259/2 = 129.5. Then, multiply 129.5 by 8654: 129.5 * 8654 = 112,141.5. Next, multiply 91072 by 33: 91072 * 33 = 3,003,336. Now, add 112,141.5 and 3,003,336: 112,141.5 + 3,003,336 = 4,115,477.5. Finally, subtract 12971 from 4,115,477.5: 4,115,477.5 - 12971 = 4,005,706.5.",
    "result": 4005706.5
}


In [14]:
4113098.0 - 4005706.5   #Margin of error

107391.5

In [15]:
2767979.0/107391.5      #how much better

25.77465628099058